In [1]:
import sys
import json
import os
# from bs4 import BeautifulSoup
from lxml import html, etree

In [2]:
sys.version

'3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [3]:
def remove_unnecessary(tag, page):
    property_list = page.xpath('.//' + tag)
    for s in property_list:
        s.drop_tree()

In [4]:
def add_to_dict(dictionary, name, tag, page):
#     If nothing in the dictionary, add it.
    if(dictionary[name] == ''):
        dictionary[name] = []
    property_list = page.xpath('.//' + tag)

    if(len(property_list) > 0):
        for obj in property_list:
#             s = obj.text.replace('\n', ' ').replace('\t', ' ')
#             s.strip()
            s = obj.text_content()
            s = s.strip()
            s = s.replace('\n', '')
            s = s.replace('\t', '')
            if(len(s) > 0):
                dictionary[name].append(s)
            obj.drop_tree()

In [5]:
def parse_data(page, url):
    # Script and Style are meaningless
    remove_unnecessary('script', page)
    remove_unnecessary('style', page)

    # Define the dictionary content and the items in it.
    # Currently we define it as:
    '''
    Link_Name: the text show on link name, since in that link it is more relevant, we should store it here
    Paragraph: the paragraphs in the page, it is the important things
    Title: I just stored the h1, h2, h3, Strong tags here, they are more important content.
    Span: The ones in span, they are not important, I just store them and extract them later.
    Others: The other content left in the page, it should have less weight.
    '''
    content = {}
    content['Link_Name'] = []
    content['Paragraph'] = []
    content['Title'] = []
    content['Span'] = []
    content['Others'] = []
    content['url'] = url
    content['h1_h2'] = []
    content['h3_h5'] = []
    content['Strong'] = []

    add_to_dict(content, 'Link_Name', 'a', page)

    add_to_dict(content, 'Link_Name', 'li', page)

    add_to_dict(content, 'Link_Name', 'option', page)

    add_to_dict(content, 'Span', 'span', page)

    add_to_dict(content, 'Paragraph', 'p', page)

#     for tag in ('title', 'h1', 'h2', 'h3', 'Strong'):
#         add_to_dict(content, 'Title', tag, soup)
    add_to_dict(content, 'Title', 'title', page)

    add_to_dict(content, 'h1_h2', 'h1', page)
    add_to_dict(content, 'h1_h2', 'h2', page)

    add_to_dict(content, 'h3_h5', 'h3', page)
    add_to_dict(content, 'h3_h5', 'h4', page)
    add_to_dict(content, 'h3_h5', 'h5', page)

#         Here since I extract all the read text above, the rest of them will not be duplicated.
#         And if there is any BROKEN html, it will read into "Others"
    for word in page.text_content().split('\n'):
        word = word.strip()
        if(len(word) > 0):
            content['Others'].append(word)
            
    return content

In [6]:
def load_map_file(path):
    with open(path) as f:
        map_file = json.load(f)
    return map_file

In [7]:
# Initialize the parsing result
# It will delete what has alread been created!
# Don't run this one if it not necessary
def clean_result():
    map_file = load_map_file('WEBPAGES_RAW/bookkeeping.json')
    readed = {}
    for path in map_file:
        readed[path] = False
    with open('WEBPAGES_RAW/already_read.json', 'w') as f:
        json.dump(readed,f)
    for file_name in os.listdir('Data/'):
        os.remove('Data/' + file_name)

In [16]:
def read_and_write(max_time):
    total_result = {}

    with open('WEBPAGES_RAW/already_read.json') as f:
        alread_read = json.load(f)

    map_file = load_map_file('WEBPAGES_RAW/bookkeeping.json')

    i = 0
    for path in map_file:
    #     If it is readed, do not read it again.
        if(alread_read[path]):
            continue

        try:
            f = open('WEBPAGES_RAW/' + path)
#             f = open('WEBPAGES_RAW/' + path, encoding="utf-8")
        except:
            print('Something wrong with ' + path)
            continue
        else:
            raw_page = f.read().replace("encoding=\"", "") + " "
#             page = html.parse(f)
            f.close()
#             print(path)
            url = map_file[path]
            try:
                page = html.fromstring(raw_page)
            except:
                print(path + ' is None !')
                total_result[path] = {'url':url, 'Link_Name':[], 'Paragraph':[], 'Title':[], 'Span':[], 'Others':[],
                                     'h1_h2':[], 'h3_h5':[], 'Strong':[]}
            else:
                total_result[path] = parse_data(page, url)

        alread_read[path] = True
        i += 1
        if(i == max_time):
            break
    
    return total_result, alread_read

In [17]:
clean_result()

In [18]:
# dealed = 0
while(True):
    total_result, already_read = read_and_write(100)
    if(len(total_result) == 0):
        break

    data_name = str(len(os.listdir('Data/'))) + '.json'
    
#     Save data
    with open('Data/' + data_name, 'w') as f:
        json.dump(total_result, f)

#     Updata what has been readed.
    with open('WEBPAGES_RAW/already_read.json', 'w') as f:
        json.dump(already_read,f)
#     dealed += len(total_result)

4/288 is None !
49/385 is None !
60/155 is None !
60/201 is None !


In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())
    xxx = page.xpath(".//p")
    for x in xxx:
        print(html.tostring(x))

In [ ]:
f = open("WEBPAGES_RAW/1/342")

In [ ]:
s = f.read().replace("\n", "")

In [ ]:
ET.parse("WEBPAGES_RAW/1/342")

In [ ]:
page = ET.fromstring(s)

In [ ]:
soup.find_all('a')[25]

In [ ]:
with open("WEBPAGES_RAW/0/141") as f:
    soup = BeautifulSoup(f.read())
#     soup = BeautifulSoup(f.read().replace('iso-8859-1', 'utf-8'))

In [ ]:
soup.find_all('a')[21]

In [ ]:
with open("WEBPAGES_RAW/0/147", encoding = "utf-8") as f:
    k = f.read()

In [ ]:
k.replace("windows-1252", "utf-8")

In [ ]:
soup = BeautifulSoup(k)

In [ ]:
soup

In [ ]:
soup = BeautifulSoup(k)

In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())

In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())
    xxx = page.xpath(".//p")
    for x in xxx:
        print(html.tostring(x))

In [ ]:
page.xpath(".//p")

In [ ]:
property_list_reg = ".//span"

In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())

In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())
property_list_reg = ".//p"
property_lst1 = page.xpath(property_list_reg)
len(property_lst1)

In [ ]:
property_list_reg = ".//span"
property_lst2 = page.xpath(property_list_reg)
len(property_lst2)

In [ ]:
a = property_lst1[0]

In [ ]:
a.text_content()

In [ ]:
a.clear()

In [ ]:
a.drop_tag()

In [ ]:
property_list_reg = ".//span"
property_lst2 = page.xpath(property_list_reg)
len(property_lst2)

In [ ]:
def cal_len(html_page, tag):
    return(len(html_page.xpath(".//" + tag)))

In [ ]:
len(property_lst1)

In [ ]:
len(property_lst1)

In [ ]:
cal_len(page, "p")

In [ ]:
with open("WEBPAGES_RAW/0/147",encoding="utf-8") as f:
    page = html.fromstring(f.read())
property_list_reg = ".//p"
property_lst1 = page.xpath(property_list_reg)
len(property_lst1)

In [ ]:
for i in range(len(property_lst1)):
    text = property_lst1[i].text_content().strip()
    if(text != ''):
        print(text)
    property_lst1[i].drop_tree()

In [ ]:
cal_len(page, "span")

In [ ]:
a

In [ ]:
with open("WEBPAGES_RAW/0/10", encoding="utf-8") as f:
    page = html.fromstring(f.read().replace("encoding=\"UTF-8\"", ""))
property_list_reg = ".//p"
property_lst1 = page.xpath(property_list_reg)
len(property_lst1)

In [ ]:
page.findall("html")

In [ ]:
f = open('WEBPAGES_RAW/0/152')
raw_page = f.read().replace("encoding=\"", "")
f.close()
page = html.fromstring(raw_page)

total_result = parse_data(page, "Idon'tCare")

In [ ]:
total_result['Others'][3]

In [ ]:
print(raw_page)